In [15]:
import pandas as pdLib 

In [16]:

lDdata = pdLib.read_csv("LiverDiseaseDataset.csv")

# Preparing Data

In [17]:
#1.
lDdata.drop("Unnamed: 0" , axis = 1 , inplace=True) # remove the unused columns
#2.
lDdata.isna().sum()



Category     0
Age          0
Sex          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
dtype: int64

In [18]:
#3.
lDdata.fillna(lDdata.median(numeric_only=True) ,inplace=True)
lDdata.isna().sum()
#5.
#6.

Category    0
Age         0
Sex         0
ALB         0
ALP         0
ALT         0
AST         0
BIL         0
CHE         0
CHOL        0
CREA        0
GGT         0
PROT        0
dtype: int64